In [1]:
from platypus import NSGAII, Problem, Real, nondominated


def found_allocation(obiettivi, patrimonio, niter):
    def belegundu(vars):
        funct = []
        for i in range(0, len(obiettivi)):
            funct.append((obiettivi[i]["valore"]/vars[i])**(1.0/obiettivi[i]["tempo"])-1.0)
            #print i, (obiettivi[i]["valore"]/vars[i])**(1.0/obiettivi[i]["tempo"])-1.0
        funct.append(-sum(vars))
        #print "Var", vars
        #print "Vincolo", sum(vars) - patrimonio
        return funct, [sum(vars) - patrimonio, ]

    types = []
    for i in range(0, len(obiettivi)):
        #print obiettivi[i]["valore"]/(1+obiettivi[i]["alfa"])**obiettivi[i]["tempo"]
        types.append(Real(obiettivi[i]["valore"]/(1+obiettivi[i]["alfa"])**obiettivi[i]["tempo"], obiettivi[i]["valore"]))
    problem = Problem(len(obiettivi), len(obiettivi)+1, 1)
    print types
    problem.types[:] = types
    problem.constraints[:] = "<=0"
    problem.function = belegundu

    algorithm = NSGAII(problem)
    algorithm.run(niter)

    feasible_solutions = [s for s in algorithm.result if s.feasible]

    return [feasible_solutions, algorithm.result, nondominated(algorithm.result)]


def budgeting(patrimonio, flussoDiRisparmio, obiettivi, niter=1000):
    risparmio = 0
    for i in range(0, len(obiettivi)):
        if i == 0:
            risparmio += obiettivi[i]["tempo"]*flussoDiRisparmio
        else:
            risparmio += (obiettivi[i]["tempo"]-obiettivi[i-1]["tempo"])*flussoDiRisparmio
        if obiettivi[i]["valore"] - risparmio < 0:
            obiettivi[i]["valore "] = 0
            risparmio -= obiettivi[i]["valore"]
            obiettivi[i]["risparmioNecessario"] = obiettivi[i]["valore"]
        else:
            obiettivi[i]["valore"] = obiettivi[i]["valore"] - risparmio
            obiettivi[i]["risparmioNecessario"] = risparmio
            risparmio = 0
    sols = allocation(obiettivi, patrimonio, niter)
    if len(sols[0]) > 0:
        sol = sols[0][0]
        for i in range(0, len(sol.variables)):
            obiettivi[i]["investimentoNecessario"] = sol.variables[i]
            obiettivi[i]["performanceAnnualizzataIpotizzata"] = sol.objectives[i]
        info = "Soluzione trovata"
    else:
        sol = sols[1][0]
        for i in range(0, len(sol.variables)):
            obiettivi[i]["investimentoNecessario"] = sol.variables[i]
            obiettivi[i]["performanceAnnualizzataIpotizzata"] = sol.objectives[i]
        info = "Soluzione non trovata"
    sol = dict()        
    sol["info"] = info
    sol["obiettivi"] = obiettivi
    return sol



0


In [4]:
import time
import json

obiettivi = [
    {
        "valore": 200000.0,
        "tempo": 2.0,
        "alfa": 0.02,
        "necessity": 1,
    },
    {
        "valore": 1500000.0,
        "tempo": 10.0,
        "alfa": 0.08,
        "necessity": 3,
    },
    {
        "valore": 1000000.0,
        "tempo": 20.0,
        "alfa": 0.08,
        "necessity": 2,
    },
    {
        "valore": 1500000.0,
        "tempo": 30.0,
        "alfa": 0.08,
        "necessity": 4,
    },
]

for obiettivo in obiettivi:
    if obiettivo["necessity"] == 1:
        obiettivo["maxRisk"] = 0.04
    elif obiettivo["necessity"] == 2:
        obiettivo["maxRisk"] = 0.08
    elif obiettivo["necessity"] == 3:
        obiettivo["maxRisk"] = 0.14
    elif obiettivo["necessity"] == 4:
        obiettivo["maxRisk"] = 0.16
    elif obiettivo["necessity"] == 5:
        obiettivo["maxRisk"] = 0.20

patrimonio = 1500000
flussoDiRisparmio = 1000

start_time = time.time()
obi = budgeting(patrimonio, flussoDiRisparmio, obiettivi)
while obi["info"] == "Soluzione non trovata":
    print json.dumps(obi, indent=2)
    minP = 5
    trovato = False
    while minP > 0 and trovato == False:
        for i in range(0, len(obiettivi)):
            if obiettivi[i]["necessity"] == minP:
                obietivoElm = obiettivi[i]
                del obiettivi[i]
                trovato = True
                break
        minP -= 1
    obi = budgeting(patrimonio, flussoDiRisparmio, obiettivi)
    

print time.time()-start_time
val = 0
for o in obi["obiettivi"]:
    val += o["investimentoNecessario"]
print json.dumps(obi, indent=2)


[<platypus.types.Real object at 0x00000000180DE2B0>, <platypus.types.Real object at 0x00000000180DE2E8>, <platypus.types.Real object at 0x00000000180DE240>, <platypus.types.Real object at 0x00000000180DE320>]
0.498000144958
{
  "info": "Soluzione trovata", 
  "obiettivi": [
    {
      "performanceAnnualizzataIpotizzata": 0.00019276994723527174, 
      "alfa": 0.02, 
      "investimentoNecessario": 197923.6851684128, 
      "tempo": 2.0, 
      "risparmioNecessario": 2000.0, 
      "maxRisk": 0.04, 
      "valore": 198000.0, 
      "necessity": 1
    }, 
    {
      "performanceAnnualizzataIpotizzata": 0.07753069206192187, 
      "alfa": 0.08, 
      "investimentoNecessario": 707086.1488459483, 
      "tempo": 10.0, 
      "risparmioNecessario": 8000.0, 
      "maxRisk": 0.14, 
      "valore": 1492000.0, 
      "necessity": 3
    }, 
    {
      "performanceAnnualizzataIpotizzata": 0.06762729254902955, 
      "alfa": 0.08, 
      "investimentoNecessario": 267449.5340502547, 
      "tem

In [3]:
import random
import pandas as pd

solutions = pd.DataFrame()

for i in range(0,100):
    solution = dict()
    solution["id"] = i
    solution["performanceAnnualizzata"] = round(random.uniform(0.01,0.08),2)
    solution["rischio"] = round(random.uniform(0.01,0.20),2)
    solutions = solutions.append(solution, ignore_index = True)

solutions = solutions.sort_values(["performanceAnnualizzata", "rischio"], ascending=[False, True])
for x in obi["obiettivi"]:
    print "Obiettivo***********************"
    print x["performanceAnnualizzataIpotizzata"], x["maxRisk"]
    subsetSol = (solutions[(solutions["rischio"] <= x["maxRisk"]) & (solutions["performanceAnnualizzata"]>=x["performanceAnnualizzataIpotizzata"])]).copy()
    subsetSol["returnRisk"] = subsetSol["performanceAnnualizzata"]/subsetSol["rischio"]  
    print subsetSol.sort_values(["returnRisk"], ascending=[False])[0:3]
    

Obiettivo***********************
7.79594522493e-07 0.04
      id  performanceAnnualizzata  rischio  returnRisk
79  79.0                     0.04     0.01         4.0
27  27.0                     0.06     0.02         3.0
18  18.0                     0.05     0.02         2.5
Obiettivo***********************
0.00889910059306 0.08
      id  performanceAnnualizzata  rischio  returnRisk
79  79.0                     0.04     0.01         4.0
27  27.0                     0.06     0.02         3.0
18  18.0                     0.05     0.02         2.5
